# 1.Build a deep learning model to classify the mnist digits dataset with Batch Normalization.

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape((60000, 28, 28, 1)).astype('float32') / 255
x_test = x_test.reshape((10000, 28, 28, 1)).astype('float32') / 255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Function to build the model with Batch Normalization
def build_model(optimizer):
    model = models.Sequential()

    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())

    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(10, activation='softmax'))

    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

# List of optimizers to try
optimizers = ['adam', 'sgd', 'rmsprop']

# Train and evaluate the model with each optimizer
for optimizer_name in optimizers:
    print(f"\nTraining with {optimizer_name} optimizer:")
    
    model = build_model(optimizer_name)
    model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.2, verbose=2)

    # Evaluate the model on the test set
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
    print(f'Test accuracy with {optimizer_name} optimizer: {test_acc}')



Training with adam optimizer:
Epoch 1/5
750/750 - 75s - loss: 0.1211 - accuracy: 0.9646 - val_loss: 0.0503 - val_accuracy: 0.9858 - 75s/epoch - 101ms/step
Epoch 2/5
750/750 - 72s - loss: 0.0410 - accuracy: 0.9870 - val_loss: 0.0403 - val_accuracy: 0.9875 - 72s/epoch - 96ms/step
Epoch 3/5
750/750 - 72s - loss: 0.0285 - accuracy: 0.9915 - val_loss: 0.0380 - val_accuracy: 0.9888 - 72s/epoch - 96ms/step
Epoch 4/5
750/750 - 72s - loss: 0.0213 - accuracy: 0.9929 - val_loss: 0.0364 - val_accuracy: 0.9887 - 72s/epoch - 96ms/step
Epoch 5/5
750/750 - 73s - loss: 0.0175 - accuracy: 0.9946 - val_loss: 0.0381 - val_accuracy: 0.9883 - 73s/epoch - 97ms/step
Test accuracy with adam optimizer: 0.9897000193595886

Training with sgd optimizer:
Epoch 1/5
750/750 - 81s - loss: 0.2350 - accuracy: 0.9395 - val_loss: 0.1111 - val_accuracy: 0.9735 - 81s/epoch - 109ms/step
Epoch 2/5
750/750 - 71s - loss: 0.0863 - accuracy: 0.9785 - val_loss: 0.0700 - val_accuracy: 0.9813 - 71s/epoch - 94ms/step
Epoch 3/5
750/7

# 2.Build a Feed Forward Neural Network for any problems with keras tuner.

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

# Load dataset (replace this with your own dataset)
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

iris = load_iris()
X, y = iris.data, iris.target

# One-hot encode the labels
y = tf.keras.utils.to_categorical(y)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the input features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the model-building function for Keras Tuner
def build_model(hp):
    model = keras.Sequential()
    
    # Input layer
    model.add(layers.InputLayer(input_shape=(X_train.shape[1],)))
    
    # Hidden layers
    for i in range(hp.Int('num_layers', min_value=1, max_value=5, step=1)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=512, step=32),
                               activation='relu'))
        model.add(layers.Dropout(rate=hp.Float('dropout_' + str(i), min_value=0.0, max_value=0.5, step=0.1)))
    
    # Output layer
    model.add(layers.Dense(units=y_train.shape[1], activation='softmax'))
    
    # Compile the model
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

# Instantiate the Keras Tuner RandomSearch
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # Adjust as needed
    directory='tuner_results',
    project_name='iris_classification')

# Search for the best hyperparameter configuration
tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Get the best hyperparameter configuration
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the final model with the best hyperparameters
final_model = tuner.hypermodel.build(best_hps)

# Train the final model
final_model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.800000011920929

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 00m 21s
Epoch 1/50
4/4 [==============================] - 2s 154ms/step - loss: 0.6174 - accuracy: 0.7250 - val_loss: 0.3871 - val_accuracy: 0.8000
Epoch 2/50
4/4 [==============================] - 0s 32ms/step - loss: 0.3887 - accuracy: 0.7917 - val_loss: 0.2046 - val_accuracy: 0.9333
Epoch 3/50
4/4 [==============================] - 0s 31ms/step - loss: 0.2703 - accuracy: 0.8500 - val_loss: 0.1857 - val_accuracy: 0.9333
Epoch 4/50
4/4 [==============================] - 0s 29ms/step - loss: 0.2603 - accuracy: 0.9083 - val_loss: 0.1230 - val_accuracy: 0.9667
Epoch 5/50
4/4 [==============================] - 0s 30ms/step - loss: 0.1649 - accuracy: 0.9250 - val_loss: 0.0754 - val_accuracy: 1.0000
Epoch 6/50
4/4 [==============================] - 0s 30ms/step - loss: 0.0976 - accuracy: 0.9667 - val_loss: 0.0474 - val_accuracy: 1.0000
Epoch 7/50
4/4 [=======================